In [1]:
import itertools
import random
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing 
from sklearn import metrics
from sklearn.model_selection import cross_val_score

#  BenchMark

In [2]:
train_data = pd.read_excel("./数据集/train.xlsx",index_col=False)
test_data = pd.read_excel("./数据集/test_A榜.xlsx",index_col=False)
new_train_data = train_data.replace('?',np.nan) # 用np.nan替换？

In [60]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'

# predictor = TabularPredictor(label= label).fit(train_data=train_data,
# #                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3)\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
                                )

No path specified. Models will be saved in: "AutogluonModels/ag-20220501_032846\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220501_032846\learner.pkl
Saving AutogluonModels/ag-20220501_032846\predictor.pkl
Beginning AutoGluon training ...
AutoGluon wi

limit_output extension: Maximum message size of 10000 exceeded with 10020 characters

In [61]:
results = predictor.fit_summary()

Loading: AutogluonModels/ag-20220501_032846\models\KNeighborsUnif\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\RandomForestGini\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\RandomForestEntr\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\ExtraTreesGini\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\ExtraTreesEntr\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\NeuralNetFastAI\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\NeuralNetFastAI\model-internals.pkl
Loading: AutogluonModels/ag-20220501_032846\models\XGBoost\model.pkl
Loading: AutogluonModels/ag-20220501_032846\models\NeuralNetTorch\model.pkl
Loading: AutogluonM

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.952911       2.925409  109.959228                0.002025           1.576801            2       True         14
1               XGBoost   0.952399       0.017951    1.941504                0.017951           1.941504            1       True         11
2              LightGBM   0.951463       0.014953    3.506632                0.014953           3.506632            1       True          4
3         LightGBMLarge   0.950722       0.015957    8.059452                0.015957           8.059452            1       True         13
4              CatBoost   0.949586       0.009972   42.239246                0.009972          42.239246            1       True          7
5            LightGBMXT   0.949358       0.024932    4.007109                0.024932           4.

# 查看数据

## 数据分布

In [2]:
train_data = pd.read_excel("./数据集/train.xlsx",index_col=False)
test_data = pd.read_excel("./数据集/test_A榜.xlsx",index_col=False)

In [3]:
# 查看标签分布
train_data["LABEL"].value_counts(normalize=True)

0    0.75
1    0.25
Name: LABEL, dtype: float64

In [4]:
# 查看数据分布，发现有很多问号，应该也是缺失值的一种
train_data.head(5)

,CUST_UID,LABEL,AGN_CNT_RCT_12_MON,ICO_CUR_MON_ACM_TRX_TM,NB_RCT_3_MON_LGN_TMS_AGV,AGN_CUR_YEAR_AMT,AGN_CUR_YEAR_WAG_AMT,AGN_AGR_LATEST_AGN_AMT,ICO_CUR_MON_ACM_TRX_AMT,COUNTER_CUR_YEAR_CNT_AMT,...,WTHR_OPN_ONL_ICO,EMP_NBR,REG_CPT,SHH_BCK,HLD_DMS_CCY_ACT_NBR,REG_DT,LGP_HLD_CARD_LVL,OPN_TM,NB_CTC_HLD_IDV_AIO_CARD_SITU,HLD_FGN_CCY_ACT_NBR
0,2a171d461bf24a739b804c2843ad4f6e,0,?,72,425.3,?,?,?,25879985.3,2,...,B,2,1200000002,32,12,1708.45,?,416.84,?,2
1,c58d5848d18548e297963c2d9e092699,0,2282,222,?,11776572.4,?,1964626.4,14755499.8,2,...,B,1002,5000002,32,22,199.42,F,195.87,D,2
2,74e6d8179e784a039bcb5a722014f4a4,0,?,2,?,?,?,4599822.3,2,2,...,B,2,?,2,12,1135.55,F,1122,NaN,2
3,1b6a8be2c2e34de09ca00d71470ec180,1,?,?,22,?,?,?,?,2,...,A,2,500002,12,12,98.45,NaN,92,C,2
4,1974b5e0440a41128a416a8cdc8a7c16,1,?,?,?,?,?,?,?,?,...,A,2,?,2,12,?,NaN,355.87,C,2


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 50 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   CUST_UID                                 12000 non-null  object
 1   AGN_CNT_RCT_12_MON                       12000 non-null  object
 2   ICO_CUR_MON_ACM_TRX_TM                   12000 non-null  object
 3   NB_RCT_3_MON_LGN_TMS_AGV                 12000 non-null  object
 4   AGN_CUR_YEAR_AMT                         12000 non-null  object
 5   AGN_CUR_YEAR_WAG_AMT                     12000 non-null  object
 6   AGN_AGR_LATEST_AGN_AMT                   12000 non-null  object
 7   ICO_CUR_MON_ACM_TRX_AMT                  12000 non-null  object
 8   COUNTER_CUR_YEAR_CNT_AMT                 12000 non-null  object
 9   PUB_TO_PRV_TRX_AMT_CUR_YEAR              12000 non-null  object
 10  MON_12_EXT_SAM_TRSF_IN_AMT               12000 non-null  o

In [6]:
# 查看各特征的数据类型
fea_type = pd.read_excel("./数据集/特征说明.xlsx",index_col=False,header=1)

fea_type.index = fea_type["字段名称"]
fea_type.drop(columns=["字段名称"],inplace=True)

# 字符型数据
fea_type = fea_type.loc[fea_type.index!="CUST_UID"]
char_type_fea = fea_type[fea_type["字符类型"] == "字符型"].index

In [7]:
# 数值型数据
fea_type = fea_type.loc[fea_type.index!="LABEL"]
fea_type = fea_type.loc[fea_type.index!="CUR_MON_VAL_VLD_CUST_NED_HLD_YEAR_DAY_AVG"]
num_type_fea = fea_type[fea_type["字符类型"] == "数值型"].index

### 缺失值查看

In [8]:
# 原始特征缺失值分布
train_data.isnull().mean()[train_data.isnull().mean()>0]

MON_12_CUST_CNT_PTY_ID          0.570750
WTHR_OPN_ONL_ICO                0.031200
LGP_HLD_CARD_LVL                0.318000
NB_CTC_HLD_IDV_AIO_CARD_SITU    0.070025
dtype: float64

In [9]:
train_data = train_data.fillna(1)

In [10]:
# 将问题replace为np.nan值后查看各特征问号的占比
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？

# 替换后特征缺失值分布
miss_fea = new_train_data.isnull().mean()[new_train_data.isnull().mean()>0.1].index
print(new_train_data.isnull().mean()[new_train_data.isnull().mean()>0.1])

AGN_CNT_RCT_12_MON                  0.535200
AGN_CUR_YEAR_AMT                    0.563525
AGN_CUR_YEAR_WAG_AMT                0.699625
AGN_AGR_LATEST_AGN_AMT              0.489825
MON_12_TRX_AMT_MAX_AMT_PCTT         0.240000
CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT     0.503400
MON_6_50_UP_ENTR_ACT_CNT            0.165500
MON_6_50_UP_LVE_ACT_CNT             0.165500
MON_12_ACT_OUT_50_UP_CNT_PTY_QTY    0.146500
MON_12_ACT_IN_50_UP_CNT_PTY_QTY     0.146500
LGP_HLD_CARD_LVL                    0.100800
NB_CTC_HLD_IDV_AIO_CARD_SITU        0.100800
dtype: float64


### 重复值查看

In [11]:
train_data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
39995    False
39996    False
39997    False
39998    False
39999    False
Length: 40000, dtype: bool

### 更改数据类型

In [11]:
for col in fea_type.index: 
    if col != "CUR_MON_VAL_VLD_CUST_NED_HLD_YEAR_DAY_AVG":
        
        if col == "LABEL":
            new_train_data[col] = new_train_data[col].astype("category")

        elif fea_type.loc[col,"字符类型"] == "数值型": 
            new_train_data[col] = new_train_data[col].astype("float32")

        elif fea_type.loc[col,"字符类型"] == "字符型":
            new_train_data[col] == new_train_data[col].astype("string")

## 数据无量纲化

### log

In [ ]:
def data_log(num_type_fea,data,num=1):
    
    new_train_data = data 
    for col in num_type_fea:
        if min(new_train_data.loc[:,col]+num-1)>=0:
            new_train_data.loc[:,col]  = np.log(new_train_data.loc[:,col]+num)
        else:
            values = new_train_data.loc[:,col].values 
            new_values = []
            for value in values:
                if value >=0:
                    new_values.append(np.log(value+num))
                else:
                    new_values.append(-np.log(np.abs(value)+num))
            new_train_data.loc[:,col] = pd.DataFrame(new_values,columns=[col])   
    return new_train_data

### one-hot

In [ ]:
def One_Hot(char_type_fea,train_X,test_X,usetype="train"):

    from sklearn.preprocessing import OrdinalEncoder
    
    if usetype == "train":
        for i in char_type_fea:
            train_X.loc[:,i] = OrdinalEncoder().fit_transform(train_X.loc[:,i].values.reshape(-1, 1)) 
        return train_X
        
    else:
        for i in char_type_fea:
            transformer = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)\
                                    .fit(train_X.loc[:,i].values.reshape(-1, 1))
            test_X.loc[:,i] = transformer.transform(test_X.loc[:,i].values.reshape(-1,1))
        return test_X

### 特征构造

In [ ]:
def GroupbyFeature(column,train,test,newcolumn):
    
    # 使用训练集创建map用的字典，方便以后遇到新的数据后，也能进行与训练集相同的处理；
    # 相当于构建以个函数的映射关系
    dic_f = dict(train.groupby(column)['LABEL'].mean())
    
    # 建立新特征
    train[newcolumn] = train[column].map(dic_f)
    test[newcolumn] = test[column].map(dic_f)
    
    return train,test     

## 填补缺失值

### 均值填补(都使用均值填补）

In [28]:
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
new_train_data = new_train_data.fillna(new_train_data.mean())
new_train_data = new_train_data.fillna(0)
new_train_data.isnull().mean().sum()

0.0

In [29]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
# #                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3)\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
                                )

No path specified. Models will be saved in: "AutogluonModels/ag-20220503_040923\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220503_040923\learner.pkl
Saving AutogluonModels/ag-20220503_040923\predictor.pkl
Beginning AutoGluon training ...
AutoGluon wi

limit_output extension: Maximum message size of 10000 exceeded with 10022 characters

In [30]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220503_040923\models\KNeighborsUnif\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\RandomForestGini\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\RandomForestEntr\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\ExtraTreesGini\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\ExtraTreesEntr\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\XGBoost\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBMLarge\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\WeightedEnsemble_L2\model.pkl


*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.950495       3.162487  105.905785                0.002991           2.368663            2       True         12
1              LightGBM   0.949466       0.021940    4.908867                0.021940           4.908867            1       True          4
2               XGBoost   0.949283       0.034906    7.498859                0.034906           7.498859            1       True         10
3            LightGBMXT   0.948895       0.028922   12.158446                0.028922          12.158446            1       True          3
4         LightGBMLarge   0.948335       0.020921   16.988567                0.020921          16.988567            1       True         11
5              CatBoost   0.947898       0.013935   61.907583                0.013935          61.

In [33]:
new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.fillna(new_train_data.mean())
new_test_data = new_test_data.fillna(0)
new_test_data.isnull().mean().sum()

if new_test_data.isnull().mean().sum()==0:
    get_result(predictor,new_test_data,test_data)

Loading: AutogluonModels/ag-20220503_040923\models\WeightedEnsemble_L2\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBMLarge\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220503_040923\models\XGBoost\model.pkl


### 中值填补

In [62]:
new_train_data = new_train_data.fillna(new_train_data.median())

In [63]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
# #                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3)\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
                                )

No path specified. Models will be saved in: "AutogluonModels/ag-20220501_033309\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220501_033309\learner.pkl
Saving AutogluonModels/ag-20220501_033309\predictor.pkl
Beginning AutoGluon training ...
AutoGluon wi

limit_output extension: Maximum message size of 10000 exceeded with 10020 characters

In [64]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220501_033309\models\KNeighborsUnif\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\RandomForestGini\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\RandomForestEntr\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\ExtraTreesGini\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\ExtraTreesEntr\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\NeuralNetFastAI\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\NeuralNetFastAI\model-internals.pkl
Loading: AutogluonModels/ag-20220501_033309\models\XGBoost\model.pkl
Loading: AutogluonModels/ag-20220501_033309\models\NeuralNetTorch\model.pkl
Loading: AutogluonM

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.950896       0.126631  74.506413                0.001994           1.550854            2       True         14
1              LightGBM   0.950411       0.013963   2.558744                0.013963           2.558744            1       True          4
2               XGBoost   0.949590       0.018949   1.902912                0.018949           1.902912            1       True         11
3         LightGBMLarge   0.948850       0.023906   7.494290                0.023906           7.494290            1       True         13
4              CatBoost   0.947793       0.009972  45.842629                0.009972          45.842629            1       True          7
5            LightGBMXT   0.947642       0.019945   5.427488                0.019945           5.427488 

### 0填补

In [171]:
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
new_train_data = new_train_data.fillna(0)

In [ ]:
for col in num_type_fea:
    if min(new_train_data.loc[:,col])>=0:
        new_train_data.loc[:,col]  = np.log(new_train_data.loc[:,col]+1)
    else:
        values = new_train_data.loc[:,col].values 
        new_values = []
        for value in values:
            if value >=0:
                new_values.append(np.log(value+1))
            else:
                new_values.append(-np.log(np.abs(value)+1))
        new_train_data.loc[:,col] = pd.DataFrame(new_values,columns=[col])   

In [172]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
# #                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3)\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
                                )

No path specified. Models will be saved in: "AutogluonModels/ag-20220502_044536\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220502_044536\learner.pkl
Saving AutogluonModels/ag-20220502_044536\predictor.pkl
Beginning AutoGluon training ...
AutoGluon wi

limit_output extension: Maximum message size of 10000 exceeded with 10023 characters

In [173]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220502_044536\models\KNeighborsUnif\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\RandomForestGini\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\RandomForestEntr\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\ExtraTreesGini\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\ExtraTreesEntr\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\NeuralNetFastAI\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\NeuralNetFastAI\model-internals.pkl
Loading: AutogluonModels/ag-20220502_044536\models\XGBoost\model.pkl
Loading: AutogluonModels/ag-20220502_044536\models\NeuralNetTorch\model.pkl
Loading: AutogluonM

*** Summary of fit() ***
Estimated performance of each model:
                  model     score_val  pred_time_val        fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  0.9537147733   4.5961866379  110.5467956066            0.0029900074       2.3345947266            2       True         14
1               XGBoost  0.9526523733   0.0329122543    3.5310223103            0.0329122543       3.5310223103            1       True         11
2              LightGBM  0.9522948267   0.0229382515    7.7104563713            0.0229382515       7.7104563713            1       True          4
3         LightGBMLarge  0.9502796800   0.0229384899   17.0755836964            0.0229384899      17.0755836964            1       True         13
4            LightGBMXT  0.9489318400   0.0370810032   13.2984035015            0.0370810032      13.2984035015            1       True          3
5              CatBoost  0.9480793600   0.0149598122   3

### 数值型数据：算法填补

In [163]:
def regessor_fillna(char_fea
                    ,X
                    ,new_data 
                    ,sortindex
                    ,model
                    ,usetype="train"
                   ):
    char_type_fea = char_fea
    X_missing_reg = X 
    rfc = model     
    
    if usetype == "train":
        for i in char_type_fea:
            X_missing_reg.loc[:,i] = OrdinalEncoder().fit_transform(X_missing_reg.loc[:,i].values.reshape(-1, 1))  
    else:
        for i in char_type_fea:
            transformer = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)\
                                    .fit(X_missing_reg.loc[:,i].values.reshape(-1, 1))
            new_data.loc[:,i] = transformer.transform(new_data.loc[:,i].values.reshape(-1,1))
    
    print("开始填充")
    for i in sortindex:

        if i not in char_type_fea:

            #构建我们的新特征矩阵和新标签
            df = X_missing_reg
            fillc = df.loc[:,i]
            df = df.loc[:,df.columns != i]

            #在新特征矩阵中，对含有缺失值的列，进行0的填补
            df_0 =SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=-1).fit_transform(df)

            #在训练集中找出用于填补的训练集和测试集
            Ytrain = fillc[fillc.notnull()]
            Xtrain = df_0[Ytrain.index,:]
            
            if usetype=="train":
                Ytest = fillc[fillc.isnull()]
                Xtest = df_0[Ytest.index,:]
            else:
                df = new_data
                fillc = df.loc[:,i]
                df = df.loc[:,df.columns != i]
                df_0 =SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=-1).fit_transform(df)
                Ytest = fillc[fillc.isnull()]
                Xtest = df_0[Ytest.index,:]
                    

            #用回归来填补训练集缺失值
            rfc = rfc.fit(Xtrain, Ytrain)
            Ypredict = rfc.predict(Xtest)

            #将填补好的特征返回到我们的原始的训练集特征矩阵中
            X_missing_reg.loc[X_missing_reg.loc[:,i].isnull(),i] = Ypredict
            new_data.loc[new_data.loc[:,i].isnull(),i] = Ypredict                            
            
    print("完成填充")
    return new_data

In [169]:
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
y = new_train_data["LABEL"]
X_missing = new_train_data.drop(columns=["LABEL","CUST_UID"])
# X_missing = new_train_data

In [170]:
# 对数值型特征使用回归算法填补

from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder

# 所使用的算法模型
model = LGBMRegressor()
# model = XGBRegressor()
# model = RandomForestRegressor() 

X_missing_reg = X_missing.copy()
sortindex = np.argsort(X_missing_reg.isnull().sum(axis=0)).index

new_train_data = regessor_fillna(char_fea=char_type_fea
                                 ,X=X_missing_reg
                                 ,new_data=new_train_data
                                 ,sortindex=sortindex
                                 ,model=model 
                                 ,usetype="train"
                                )
new_train_data = new_train_data.fillna(-1)

开始填充
完成填充


In [171]:
new_train_data = data_log(num_type_fea,new_train_data,num=1)

In [172]:
Folder_Path = "./数据集/缺失值处理/"
if not os.path.exists(Folder_Path):
    os.makedirs(Folder_Path)
new_train_data.to_excel(Folder_Path+"fillna_regressor_train_data.xlsx",index=False)
# y.to_excel(Folder_Path+"train_y.xlsx",index=False)

In [173]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
# #                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3)\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
#                                     ,hyperparameters = 'multimodal',num_stack_levels=1,num_bag_folds=5 
                                )

No path specified. Models will be saved in: "AutogluonModels/ag-20220505_021827\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220505_021827\learner.pkl
Saving AutogluonModels/ag-20220505_021827\predictor.pkl
Beginning AutoGluon training ...
AutoGluon wi

		Features can also be unused if they carry very little information, such as being categorical but having almost entirely unique values or being duplicates of other features.
		These features do not need to be present at inference time.
		('object', [])                     : 1 | ['CUST_UID']
		('object', ['datetime_as_object']) : 1 | ['LGP_HLD_CARD_LVL']
	Types of features in original data (exact raw dtype, raw dtype):
		('float64', 'float') : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', 'object') :  3 | ['MON_12_CUST_CNT_PTY_ID', 'WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
	Types of features in original data (raw dtype, special dtypes):
		('float', [])                      : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', [])                     :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_

limit_output extension: Maximum message size of 10000 exceeded with 10024 characters

In [174]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220505_021827\models\KNeighborsUnif\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\KNeighborsDist\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\LightGBMXT\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\RandomForestGini\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\RandomForestEntr\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\CatBoost\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\ExtraTreesGini\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\ExtraTreesEntr\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\NeuralNetFastAI\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\NeuralNetFastAI\model-internals.pkl
Loading: AutogluonModels/ag-20220505_021827\models\XGBoost\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\NeuralNetTorch\model.pkl
Loading: AutogluonM

*** Summary of fit() ***
Estimated performance of each model:
                  model     score_val  pred_time_val        fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2  0.9526310400   0.3292894363  214.1313459873            0.0039889812       3.0558264256            2       True         14
1               XGBoost  0.9516309333   0.1487734318    8.5378367901            0.1487734318       8.5378367901            1       True         11
2              LightGBM  0.9511044267   0.0249333382    5.7974925041            0.0249333382       5.7974925041            1       True          4
3              CatBoost  0.9489344000   0.0179486275   64.8005261421            0.0179486275      64.8005261421            1       True          7
4         LightGBMLarge  0.9488209067   0.0219414234   16.1737334728            0.0219414234      16.1737334728            1       True         13
5            LightGBMXT  0.9482056533   0.0478715897   1

### 选择性填充与删除

# 模型集成

## best_quality

In [16]:
# 训练数据缺失值处理
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
new_train_data.dropna(how="all",inplace=True)
new_train_data = new_train_data.fillna(0)
new_train_data = data_log(num_type_fea,new_train_data,num=1)

In [17]:
# 测试数据缺失值处理
test_data.isnull().mean()[test_data.isnull().mean()>0]
test_data = test_data.fillna(1)

new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.fillna(0)
new_test_data = data_log(num_type_fea,new_test_data,num=1)

In [18]:
# # 特征构造
# for col in char_type_fea:
#     new_col = col+'_ratio'
#     new_train_data,new_test_data = GroupbyFeature(col,new_train_data,new_test_data,new_col) 

In [20]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
#                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3
#                              ,sample_weight = "balance_weight"
                             ,sample_weight = "auto_weight"   
                            )\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
#                                     ,hyperparameters = 'multimodal'                                  
                                    , presets='best_quality'
                                    , time_limit=3600*5
                                )

auto_weight currently does not use any sample weights.
No path specified. Models will be saved in: "AutogluonModels/ag-20220507_061239\"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}, 'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220507_061239\lea

		These features do not need to be present at inference time.
		('object', []) : 1 | ['CUST_UID']
	Types of features in original data (exact raw dtype, raw dtype):
		('float64', 'float') : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', 'object') :  4 | ['MON_12_CUST_CNT_PTY_ID', 'WTHR_OPN_ONL_ICO', 'LGP_HLD_CARD_LVL', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
	Types of features in original data (raw dtype, special dtypes):
		('float', [])                      : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', [])                     :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('object', ['datetime_as_object']) :  2 | ['MON_12_CUST_CNT_PTY_ID', 'LGP_HLD_CARD_LVL']
	Types of features in processed data (exact raw dtype, raw dtype):
		('category', 'category') :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CT

limit_output extension: Maximum message size of 10000 exceeded with 10038 characters

In [21]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220507_061239\models\KNeighborsUnif_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\KNeighborsDist_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\LightGBMXT_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\LightGBM_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\RandomForestGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\RandomForestEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\CatBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\ExtraTreesGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\ExtraTreesEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\NeuralNetFastAI_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\XGBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\NeuralNetTorch_BAG_L1\model.pkl
Loading: AutogluonMode

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.952544     130.472257  12718.894662                0.009973           9.667130            3       True         26
1       WeightedEnsemble_L2   0.952102      14.676834   5884.731272                0.008978          11.517459            2       True         14
2         LightGBMXT_BAG_L2   0.951957     120.278841  10262.750093                0.661164          89.414819            2       True         15
3      LightGBMLarge_BAG_L2   0.951891     120.292821  10523.148702                0.675144         349.813427            2       True         25
4           LightGBM_BAG_L2   0.951879     120.238951  10258.348760                0.621274          85.013486            2       True         16
5           CatBoost_BAG_L2   0.951816     120.160300  10632.3

In [22]:
# save_path = r"F:\Anaconda\Kaggle\招商银行\AutogluonModels\ag-20220506_032511"
# predictor = TabularPredictor.load(save_path) 

In [23]:
get_result(predictor,new_test_data,test_data)

Loading: AutogluonModels/ag-20220507_061239\models\WeightedEnsemble_L3\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\CatBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\ExtraTreesEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\ExtraTreesGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\KNeighborsDist_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\KNeighborsUnif_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\LightGBMLarge_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\LightGBMXT_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\LightGBM_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\NeuralNetFastAI_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\NeuralNetTorch_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220507_061239\models\RandomForestEntr_BAG_L1\model.pkl
Loading: AutogluonMo

## multimodal+stacking

In [27]:
# 训练数据缺失值处理
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
new_train_data.dropna(how="all",inplace=True)
new_train_data = new_train_data.fillna(0)
new_train_data = new_train_data.drop(columns=["CUST_UID"])
new_train_data = data_log(num_type_fea,new_train_data,num=1)

# 测试数据缺失值处理
test_data.isnull().mean()[test_data.isnull().mean()>0]
test_data = test_data.fillna(1)

new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.fillna(0)
new_test_data = new_test_data.drop(columns=["CUST_UID"])
new_test_data = data_log(num_type_fea,new_test_data,num=1)

In [28]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
#                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3
#                              ,sample_weight = "balance_weight"
                             ,sample_weight = "auto_weight"   
                            )\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
#                                     ,hyperparameters = 'multimodal'                                  
                                    , presets='best_quality'
                                    , time_limit=3600*5
                                )
get_result(predictor,new_test_data,test_data)

auto_weight currently does not use any sample weights.
No path specified. Models will be saved in: "AutogluonModels/ag-20220509_010141\"
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}, 'auto_stack': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': None,
 'num_bag_sets': None,
 'num_stack_levels': None,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220509_010141\lea

	Types of features in original data (raw dtype, special dtypes):
		('float', [])                      : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', [])                     :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('object', ['datetime_as_object']) :  2 | ['MON_12_CUST_CNT_PTY_ID', 'LGP_HLD_CARD_LVL']
	Types of features in processed data (exact raw dtype, raw dtype):
		('category', 'category') :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('float64', 'float')     : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('int64', 'int')         :  2 | ['MON_12_CUST_CNT_PTY_ID', 'LGP_HLD_CARD_LVL']
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])             :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('float', [])          

limit_output extension: Maximum message size of 10000 exceeded with 10037 characters

In [29]:
results = predictor.fit_summary(show_plot=True)

Loading: AutogluonModels/ag-20220509_010141\models\KNeighborsUnif_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\KNeighborsDist_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\LightGBMXT_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\LightGBM_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\RandomForestGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\RandomForestEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\CatBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\ExtraTreesGini_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\ExtraTreesEntr_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\NeuralNetFastAI_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\XGBoost_BAG_L1\model.pkl
Loading: AutogluonModels/ag-20220509_010141\models\NeuralNetTorch_BAG_L1\model.pkl
Loading: AutogluonMode

*** Summary of fit() ***
Estimated performance of each model:
                      model     score_val   pred_time_val          fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3  0.9524625200  135.0573639870  12796.1242277622            0.0099732876      10.0263574123            3       True         26
1       WeightedEnsemble_L2  0.9521019417   15.2375683784   5879.3410763741            0.0089755058      11.8373515606            2       True         14
2         LightGBMXT_BAG_L2  0.9519995883  122.7544331551  10279.7340404987            0.6911385059      91.4156069756            2       True         15
3           LightGBM_BAG_L2  0.9519085367  122.7135260105  10275.2957153320            0.6502313614      86.9772818089            2       True         16
4           CatBoost_BAG_L2  0.9518853483  122.6278502941  10670.0767242908            0.5645556450     481.7582907677            2       True         19
5      LightGB

### modal 

In [26]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = "LABEL"
eval_metric = 'roc_auc'
# predictor = TabularPredictor(label= label).fit(train_data=train_data,
#                                                num_bag_folds=5, num_bag_sets=1, num_stack_levels=2,
#                                                ag_args_fit={'num_gpus': 0})

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3
                            ,sample_weight = "balance_weight")\
                            .fit(
                                    new_train_data
                                    ,ag_args_fit={'num_gpus': 1}
                                    ,hyperparameters = 'multimodal',num_stack_levels=1,num_bag_folds=5                                   
#                                     , presets='best_quality'
#                                     , time_limit=3600*13
                                )

get_result(predictor,new_test_data,test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20220507_152028\"
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1}, 'num_bag_folds': 5, 'num_stack_levels': 1}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': None,
 'num_stack_levels': 1,
 'pseudo_data': None,
 'quantile_levels': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Saving AutogluonModels/ag-20220507_152028\learner.pkl
Saving AutogluonModels/ag-20220507_152028\predictor.pkl
Using pr

	Types of features in original data (exact raw dtype, raw dtype):
		('float64', 'float') : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', 'object') :  4 | ['MON_12_CUST_CNT_PTY_ID', 'WTHR_OPN_ONL_ICO', 'LGP_HLD_CARD_LVL', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
	Types of features in original data (raw dtype, special dtypes):
		('float', [])                      : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_ACM_TRX_TM', 'NB_RCT_3_MON_LGN_TMS_AGV', 'AGN_CUR_YEAR_AMT', 'AGN_CUR_YEAR_WAG_AMT', ...]
		('object', [])                     :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('object', ['datetime_as_object']) :  2 | ['MON_12_CUST_CNT_PTY_ID', 'LGP_HLD_CARD_LVL']
	Types of features in processed data (exact raw dtype, raw dtype):
		('category', 'category') :  2 | ['WTHR_OPN_ONL_ICO', 'NB_CTC_HLD_IDV_AIO_CARD_SITU']
		('float64', 'float')     : 45 | ['AGN_CNT_RCT_12_MON', 'ICO_CUR_MON_AC

limit_output extension: Maximum message size of 10000 exceeded with 10005 characters

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
get_result(predictor,new_test_data,test_data)

## 自定义堆叠

In [45]:
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from combo.models import score_comb 
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import pickle 
import joblib

In [46]:
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
X = new_train_data.drop(columns=["LABEL","CUST_UID"])
y = new_train_data["LABEL"]
X = One_Hot(char_type_fea,train_X=X
            ,test_X=None
            ,usetype="train") 
X = X.fillna(-1)
X = data_log(num_type_fea,X,num=2)

new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.drop(columns=["CUST_UID"])
new_test_data = One_Hot(char_type_fea,train_X=X
                        ,test_X=new_test_data
                        ,usetype="test")
new_test_data = new_test_data.fillna(-1)
new_test_data = data_log(num_type_fea,new_test_data,num=2)

### 先堆叠再平均

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import math 

random = 407
pred_probs = pd.DataFrame()
for i in range(100):
    
    random = random + i 
    
    clf1 = XGBClassifier(
        n_estimators = 100 + math.floor(random/5),
        max_depth = 3 + math.floor(random/300),
        min_child_weight = 1 + math.floor(random/400),
        eval_metric='mlogloss',random_state=random)
    clf2 = LGBMClassifier(
        n_estimators= 100 + math.floor(random/5),
        max_depth = 4 + math.floor(random/300),
        num_leaves = 10 + math.floor(random/200),
        random_state=random)
    clf3 = CatBoostClassifier(
        n_estimators= 100 + math.floor(random/5),
        learning_rate=0.01,
        depth = 4 + math.floor(random/300),
        silent=True,random_state=random)
    clf4 = KNeighborsClassifier(
    n_neighbors=1 + math.floor(random/400)
                                )

    meta_model = MLPClassifier(activation = 'tanh',random_state=random)
    # meta_model = LogisticRegression()
    cv_num = 3
    sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3,clf4
                                            ] 
                             ,meta_classifier=meta_model
                             ,use_probas=True
                             ,store_train_meta_features = True
                             ,cv=cv_num
                             ,random_state=random
    #                          ,fit_base_estimators=False
                             )

    
    skf = StratifiedKFold(n_splits=5
                          ,shuffle=True
                          ,random_state=random)

    roc = []
    pred_prob = []
    for train_index, test_index in skf.split(X, y):
        train_X, test_X = X.iloc[train_index,:], X.iloc[test_index,:]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
#         train_X.index = range(train_X.shape[0]) 
#         train_y.index =range(train_y.shape[0]) 
#         test_X.index = range(test_X.shape[0])
#         test_y.index = range(test_y.shape[0]) 

        sclf.fit(train_X,train_y)
        auc = roc_auc_score(test_y, sclf.predict_proba(test_X)[:, 1])
        roc.append(auc)
        pred_prob.append(sclf.predict_proba(new_test_data)[:, 1])
    
    pred_prob = pd.DataFrame(np.mean(np.array(pred_prob),axis=0)
                             ,columns=["StackingCV"])
    pred_probs = pd.concat([pred_probs,pred_prob],axis=1)

    print("第{a}轮StackingCV线下ROC：".format(a=i+1),roc)
    print("第{a}轮StackingCV线下ROC：".format(a=i+1),np.mean(roc))
    print("*"*50)

第1轮StackingCV线下ROC： [0.9478250416666667, 0.9477921666666667, 0.9512869583333334, 0.9497621666666668, 0.947717625]
第1轮StackingCV线下ROC： 0.9488767916666667
**************************************************
第2轮StackingCV线下ROC： [0.94937275, 0.9503717916666667, 0.9446195833333333, 0.9495035, 0.9508687083333334]
第2轮StackingCV线下ROC： 0.9489472666666667
**************************************************
第3轮StackingCV线下ROC： [0.9456852916666667, 0.9491201666666667, 0.9505370833333333, 0.9482251666666668, 0.9502675416666667]
第3轮StackingCV线下ROC： 0.9487670500000001
**************************************************
第4轮StackingCV线下ROC： [0.945707, 0.9493045833333333, 0.953247875, 0.9478559166666665, 0.9477972916666667]
第4轮StackingCV线下ROC： 0.9487825333333333
**************************************************
第5轮StackingCV线下ROC： [0.9447268333333333, 0.9470182083333334, 0.95071075, 0.9505665000000001, 0.9503375]
第5轮StackingCV线下ROC： 0.9486719583333333
**************************************************
第6轮

第41轮StackingCV线下ROC： [0.9480854166666667, 0.9485437916666667, 0.9481090833333334, 0.9527997499999998, 0.9500298333333332]
第41轮StackingCV线下ROC： 0.9495135749999999
**************************************************
第42轮StackingCV线下ROC： [0.9473690833333333, 0.9526470833333335, 0.9492510000000001, 0.9521353333333334, 0.9466637083333334]
第42轮StackingCV线下ROC： 0.9496132416666667
**************************************************
第43轮StackingCV线下ROC： [0.9564291666666667, 0.9491801666666665, 0.9520019999999999, 0.9461442083333335, 0.9450495416666668]
第43轮StackingCV线下ROC： 0.9497610166666666
**************************************************
第44轮StackingCV线下ROC： [0.9499897500000001, 0.949074625, 0.9490365416666666, 0.9530485, 0.9467545833333332]
第44轮StackingCV线下ROC： 0.9495808
**************************************************
第45轮StackingCV线下ROC： [0.9528117500000002, 0.9497942916666665, 0.951213, 0.9490989583333334, 0.9464824166666665]
第45轮StackingCV线下ROC： 0.9498800833333334
*********************

limit_output extension: Maximum message size of 10000 exceeded with 10109 characters

In [ ]:
pd.set_option('precision',10) #显示3位
y_predproba = pred_probs.mean(axis=1)
submit = pd.concat([test_data["CUST_UID"],y_predproba],axis=1)

In [ ]:
from datetime import datetime
now = datetime.now()
day_now = datetime.strftime(now,'%Y-%m-%d')
time_now = datetime.strftime(now,"%H_%M_%S")

Folder_path = "./testA/{a}/".format(a=day_now)
if not os.path.exists(Folder_path):
    os.makedirs(Folder_path)
submit.to_csv(Folder_path+"submission{time}.txt".format(time=time_now)
              ,index=False,header=False,encoding='utf-8',sep='\t',float_format='%.10f') # 保存的历史文件

### 先训练在堆叠

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

random = 407
clf_list = []
for i in range(100):
    
    random = random + i 
    
    clf1 = XGBClassifier(
        n_estimators = 100 + math.floor(random/5),
        max_depth = 3 + math.floor(random/300),
        min_child_weight = 1 + math.floor(random/400),
        eval_metric='mlogloss',random_state=random)
    clf2 = LGBMClassifier(
        n_estimators= 100 + math.floor(random/5),
        max_depth = 4 + math.floor(random/300),
        num_leaves = 10 + math.floor(random/200),
        random_state=random)
    clf3 = CatBoostClassifier(
        n_estimators= 100 + math.floor(random/5),
        learning_rate=0.01,
        depth = 4 + math.floor(random/300),
        silent=True,random_state=random)
    clf4 = KNeighborsClassifier(
    n_neighbors=1 + math.floor(random/400)
                                )

    
    clf_list.append(clf1)
    clf_list.append(clf2)
    clf_list.append(clf3)
    clf_list.append(clf4)

meta_model = MLPClassifier(activation = 'tanh',random_state=random)
# meta_model = LogisticRegression()
cv_num = 3
sclf = StackingCVClassifier(classifiers=clf_list 
                         ,meta_classifier=meta_model
                         ,use_probas=True
                         ,store_train_meta_features = True
                         ,cv=cv_num
                         ,random_state=random
#                          ,fit_base_estimators=False
                         )

skf = StratifiedKFold(n_splits=5
                      ,shuffle=True
                      ,random_state=random)

roc = []
pred_prob = []
num_k = 0 
for train_index, test_index in skf.split(X, y):  
    
    train_X, test_X = X.iloc[train_index,:], X.iloc[test_index,:]
    train_y, test_y = y.iloc[train_index], y.iloc[test_index]

    sclf.fit(train_X,train_y)
    auc = roc_auc_score(test_y, sclf.predict_proba(test_X)[:, 1])
    roc.append(auc)
    pred_prob.append(sclf.predict_proba(new_test_data)[:, 1])
    
    num_k = num_k+1 
    print("第{a}轮交叉验证结束".format(a=num_k))

pred_prob = pd.DataFrame(np.mean(np.array(pred_prob),axis=0)
                             ,columns=["StackingCV"])

print("StackingCV线下ROC：",roc)
print("StackingCV线下ROC：",np.mean(roc))
print("*"*50)

In [ ]:
y_predproba = pred_prob
submit = pd.concat([test_data["CUST_UID"],y_predproba],axis=1)

In [ ]:
from datetime import datetime
now = datetime.now()
day_now = datetime.strftime(now,'%Y-%m-%d')
time_now = datetime.strftime(now,"%H_%M_%S")

Folder_path = "./testA/{a}/".format(a=day_now)
if not os.path.exists(Folder_path):
    os.makedirs(Folder_path)
submit.to_csv(Folder_path+"submission{time}.txt".format(time=time_now)
              ,index=False,header=False,encoding='utf-8',sep='\t',float_format='%.10f') # 保存的历史文件

# 预测结果

## 测试集缺失值填补

In [114]:
new_train_data = train_data.replace('?',np.nan)#用np.nan替换？
X = new_train_data.drop(columns=["LABEL","CUST_UID"])
y = new_train_data["LABEL"]
X = One_Hot(char_type_fea,train_X=X
            ,test_X=None
            ,usetype="train") 
X = X.fillna(-1)
X = data_log(num_type_fea,X,num=2)

new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.drop(columns=["CUST_UID"])
new_test_data = One_Hot(char_type_fea,train_X=X
                        ,test_X=new_test_data
                        ,usetype="test")
new_test_data = new_test_data.fillna(-1)
new_test_data = data_log(num_type_fea,new_test_data,num=2)

In [115]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
skf = StratifiedKFold(n_splits=5
                      ,shuffle=True
                      ,random_state=407)

pred_probs = {}
for clf, label in zip([clf1, clf2, clf3,clf4, sclf], 
                          ['XGB', 
                           'LGBM', 
                           'Cat',
                           'KNN',
                           'StackingCV']):
    
    roc = []
    pred_prob = []
    for train_index, test_index in skf.split(X, y):
        train_X, test_X = X.iloc[train_index,:], X.iloc[test_index,:]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        clf.fit(train_X,train_y)
        auc = roc_auc_score(test_y, clf.predict_proba(test_X)[:, 1])
        roc.append(auc)
        pred_prob.append(clf.predict_proba(new_test_data)[:, 1])
        
    pred_probs[label] = pred_prob
    
    print("{a}线下ROC：".format(a=label),roc)
    print("{a}线下ROC：".format(a=label),np.mean(roc))
    print("*"*50)

XGB线下ROC： [0.9441488333333334, 0.9449455000000001, 0.9494190416666665, 0.9478399166666667, 0.9433204166666667]
XGB线下ROC： 0.9459347416666667
**************************************************
LGBM线下ROC： [0.9489535416666666, 0.949116125, 0.9518795833333333, 0.9508032083333333, 0.9478317500000001]
LGBM线下ROC： 0.9497168416666666
**************************************************
Cat线下ROC： [0.9497739583333332, 0.9497801666666668, 0.9523079583333334, 0.951830875, 0.9484618749999999]
Cat线下ROC： 0.9504309666666666
**************************************************
KNN线下ROC： [0.7271356666666666, 0.7375070416666667, 0.7307043333333334, 0.739273625, 0.74134925]
KNN线下ROC： 0.7351939833333333
**************************************************
StackingCV线下ROC： [0.9501445833333333, 0.9504121666666666, 0.9531057916666665, 0.95253025, 0.9489845416666667]
StackingCV线下ROC： 0.9510354666666666
**************************************************


### 数值型：算法填补

In [175]:
y = new_train_data["LABEL"]
X_missing = new_train_data.drop(columns=["LABEL","CUST_UID"])

new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.drop(columns=["CUST_UID"])

# 算法填补缺失值
new_test_data = regessor_fillna(char_fea=char_type_fea
                                 ,X=X_missing_reg
                                 ,new_data=new_test_data
                                 ,sortindex=sortindex
                                 ,model=model 
                                 ,usetype="test"
                                )
new_test_data = new_test_data.fillna(-1)

开始填充
完成填充


In [176]:
new_test_data = data_log(num_type_fea,new_test_data,num=1)

### 0填补

In [113]:
new_test_data = test_data.replace('?',np.nan)#用np.nan替换？
new_test_data = new_test_data.fillna(0) 

## 计算并保存预测结果

In [15]:
def get_result(predictor,new_test_data,test_data):
    pd.set_option('precision', 10) #显示3位
    y_predproba = predictor.predict_proba(new_test_data, as_multiclass=False)
    submit = pd.concat([test_data["CUST_UID"],y_predproba],axis=1)

    from datetime import datetime
    now = datetime.now()
    day_now = datetime.strftime(now,'%Y-%m-%d')
    time_now = datetime.strftime(now,"%H_%M_%S")

    Folder_path = "./testA/{a}/".format(a=day_now)
    if not os.path.exists(Folder_path):
        os.makedirs(Folder_path)
    submit.to_csv(Folder_path+"submission{time}.txt".format(time=time_now)
                  ,index=False,header=False,encoding='utf-8',sep='\t',float_format='%.10f') # 保存的历史文件

In [177]:
pd.set_option('precision', 10) #显示3位
y_predproba = predictor.predict_proba(new_test_data, as_multiclass=False)

Loading: AutogluonModels/ag-20220505_021827\models\WeightedEnsemble_L2\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\LightGBM\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\NeuralNetTorch\model.pkl
Loading: AutogluonModels/ag-20220505_021827\models\XGBoost\model.pkl


In [178]:
submit = pd.concat([test_data["CUST_UID"],y_predproba],axis=1)

In [179]:
from datetime import datetime
now = datetime.now()
day_now = datetime.strftime(now,'%Y-%m-%d')
time_now = datetime.strftime(now,"%H_%M_%S")

Folder_path = "./testA/{a}/".format(a=day_now)
if not os.path.exists(Folder_path):
    os.makedirs(Folder_path)
submit.to_csv(Folder_path+"submission{time}.txt".format(time=time_now)
              ,index=False,header=False,encoding='utf-8',sep='\t',float_format='%.10f') # 保存的历史文件

In [16]:
 # 当天用于提交的结果
submit.to_csv(Folder_path+"submission.txt"
              ,index=False,header=False,encoding='utf-8',sep='\t',float_format='%.10f')

## 判断是否符合输出格式

In [275]:
# 检测文件是否是utf-8无bom格式的
import sys, codecs
 
def detectUTF8(file_name):
    state = 0
    line_num = 0
    file_obj = open(file_name)
    all_lines = file_obj.readlines()
    file_obj.close()
    for line in all_lines:
        line_num += 1
        line_len = len(line)
        for index in range(line_len):
            if state == 0:
                if ord(line[index]) & 0x80 == 0x00:  # 上表中的第一种情况
                    state = 0
                elif ord(line[index]) & 0xE0 == 0xC0:  # 上表中的第二种情况
                    state = 1
                elif ord(line[index]) & 0xF0 == 0xE0:  # 第三种
                    state = 2
                elif ord(line[index]) & 0xF8 == 0xF0:  # 第四种
                    state = 3
                else:
                    print("%s isn't a utf8 file,line:\t" % file_name + str(line_num))
                    sys.exit(1)
            else:
                if not ord(line[index]) & 0xC0 == 0x80:
                    print("%s isn't a utf8 file in line:\t" % file_name + str(line_num))
                    sys.exit(1)
                state -= 1
                
    if existBOM(file_name):
        print("%s isn't a standard utf8 file,include BOM header." % file_name)
        sys.exit(1)
    else:
        print("格式正确")
    
    
def existBOM(file_name):
    file_obj = open(file_name, 'r')
    code = file_obj.read(3)
    file_obj.close()
    if code == codecs.BOM_UTF8:  # 判断是否包含EF BB BF
        return True
    return False
 
if __name__ == "__main__":
    file_name = Folder_path + 'submission.txt'
    detectUTF8(file_name)

格式正确
